In [2]:
import os.path
import os
import igraph as ig
import pathlib
import pickle
import time
import pandas as pd
import numpy as np
from multiprocessing import Pool
from joblib import Parallel, delayed
from tqdm.notebook import trange, tqdm

Specification of data and supplementary dirs

In [3]:
# data_type = '2009-2010'
data_type = '2000-2001'
# data_type = '1999-2000'
# data_type = 'subsample'
DATA_PATH = f'../data arxiv/hyperedges_list_arxiv_{data_type}.txt'
LINEGRAPH_EDGES_PATH = f'../suplementary/linegraphs/linegraph_weighted_edges_list_{data_type}.txt'
LINEGRAPH_NODES_PATH = f'../suplementary/linegraphs/linegraph_weighted_nodes_list_{data_type}.txt'
DIST_DIR = "../suplementary/dist"
CLIQUE_PATH = f"../suplementary/clique-projections/{data_type}-proj-graph.txt"

### Data reading

In [4]:
data_path = DATA_PATH
with open(data_path) as file:
    line = file.readline()

if data_type in ["subsample", "test"]:
    hyperedges_list = line.split('], ')
    hyperedges_list = [x.replace('[', '') for x in hyperedges_list]
    hyperedges_list = [x.replace(']', '') for x in hyperedges_list]  # removes the last "]""
else:
    hyperedges_list = line.split(']", ')
    hyperedges_list = [x.replace('"[', '') for x in hyperedges_list]
    hyperedges_list = [x.replace(']"', '') for x in hyperedges_list]  # removes the last "]""

print(len(hyperedges_list))
print(hyperedges_list[0])
print(hyperedges_list[-1])

8914
'hep-ex', 'physics.comp-ph'
'math.ag', 'math.cv'


In [5]:
hyperedges_list, counts_list = np.unique(hyperedges_list,
                                         return_counts=True)  # counts_list contains info about frequencies of each hyperedge
print(len(hyperedges_list))
print(len(counts_list))

hyperedges_list = [x.replace("'", '') for x in hyperedges_list]
hyperedges_list = [x.split(', ') for x in hyperedges_list]
print(hyperedges_list[:3])
print(counts_list[:3])

2205
2205
[['astro-ph', 'astro-ph.ga'], ['astro-ph', 'cond-mat'], ['astro-ph', 'cond-mat', 'gr-qc', 'hep-ph']]
[1 1 2]


In [6]:
# left hyperedges of size 1
hyperedges_list_ids = [i for i in range(len(hyperedges_list)) if len(hyperedges_list[i]) > 1]
print(len(hyperedges_list_ids))
if len(hyperedges_list_ids) < len(hyperedges_list):
    hyperedges_list = [hyperedges_list[i] for i in hyperedges_list_ids]
    counts_list = [counts_list[i] for i in hyperedges_list_ids]

2205


In [7]:
# create supplementary dictionary containing nodes as keys and hyperedges to which they are incident as values. It helps to access needed hyperedges faster
def get_nodes_to_edges(edges_to_nodes_dict):
    nodes_to_edges_dict = {}
    for edge, nodes in edges_to_nodes_dict.items():
        for node in nodes:
            if node in nodes_to_edges_dict.keys():
                nodes_to_edges_dict[node].append(edge)
            else:
                nodes_to_edges_dict[node] = [edge]
    return nodes_to_edges_dict

In [8]:
edges_to_nodes_dict = {i: hyperedges_list[i] for i in
                       range(len(hyperedges_list))}  # key = hyperedge, value = nodes in hyperedge
edges_to_counts_dict = {i: counts_list[i] for i in range(len(counts_list))}
nodes_to_edges_dict = get_nodes_to_edges(
    edges_to_nodes_dict)  # key = node, value = list of hyperedges in which the node participates
print(f'nodes count = {len(nodes_to_edges_dict.keys())}')

nodes count = 135


### Line-graph construction functions

In [9]:
# weight_function is needed for creation of weighted line graph, w1 (w2) is e1's (e2's) weight in weighted hypergraph
def weight_function(e1, e2, w1, w2):
    if e1 == e2:
        return (1 / w1) * ((1 / 3) * (len(e1) + 1) - 1)
    union_len = len(set(e1).union(e2))
    intersect_len = len(set(e1).intersection(e2))
    return (1 / 2) * (1 / w1 + 1 / w2) * ((1 / 3) * union_len * (1 + 1 / intersect_len) - 1)


def generate_weighted_line_graph(edges_to_nodes_dict, edges_to_counts_dict):
    file_path = pathlib.Path(LINEGRAPH_EDGES_PATH)

    if not file_path.exists():

        items = list(edges_to_nodes_dict.items())
        line_nodes_ids_dict = {items[i][0]: i for i in
                               range(len(items))}  # in line-graph nodes = hyperedges in intial hypergraph
        line_edges_list = []
        line_edges_weights_list = []

        for i in range(len(items)):
            edge, nodes = items[i]
            line_edges_list.append([i, i])  # our line-graph has self-loops
            line_edges_weights_list.append(weight_function(nodes, nodes,
                                                           edges_to_counts_dict[edge],
                                                           edges_to_counts_dict[edge]))  # add weights of self-loops

        for i in tqdm(range(len(items))):
            edge, nodes = items[i]
            set_nodes = set(nodes)
            neighbors = [k for k, v in items if
                         not k == edge and
                         (len(v) >= len(nodes)) and
                         not set_nodes.isdisjoint(v)]  # find hyperedges that intersect with the current one

            if len(neighbors) > 0:
                for neighbor in neighbors:
                    line_edges_list.append([line_nodes_ids_dict[edge], line_nodes_ids_dict[
                        neighbor]])  # each intersection leads to edge in the line-graph
                    line_edges_weights_list.append(weight_function(nodes, edges_to_nodes_dict[neighbor],
                                                                   edges_to_counts_dict[edge],
                                                                   edges_to_counts_dict[neighbor]))

        G = ig.Graph(n=len(items), edges=line_edges_list,
                     edge_attrs={'weight': line_edges_weights_list},
                     vertex_attrs={'label': list(line_nodes_ids_dict.keys())})
        edges_list = G.get_edgelist()
        print('saving line graph')
        with open(LINEGRAPH_EDGES_PATH, 'w') as file:
            lines = []
            for edge_id in range(len(edges_list)):
                edge_weight = G.es[edge_id]["weight"]
                lines.append(f'{edges_list[edge_id][0]} {edges_list[edge_id][1]} {edge_weight}\n')
            file.writelines(lines)
        with open(LINEGRAPH_NODES_PATH, 'w') as file:
            file.writelines([str(G.vs[i]['label']) + '\n' for i in range(len(items))])

        print('line graph saved')
        del G

    with open(LINEGRAPH_EDGES_PATH, 'r') as file:
        lines = file.readlines()
    with open(LINEGRAPH_NODES_PATH, 'r') as file:
        lines_nodes = file.readlines()
    edges_list = []
    weights = []
    nodes = []
    print('preparing data for line graph creation')
    print(len(lines))
    for line in lines:
        from_node, to_node, weight = line.split(' ')
        from_node = int(from_node)
        to_node = int(to_node)
        if from_node not in nodes:
            nodes.append(from_node)
        if to_node not in nodes:
            nodes.append(to_node)
        weight = float(weight)
        edges_list.append((from_node, to_node))
        weights.append(weight)

    print('line graph creation')
    G = ig.Graph(n=len(nodes), edges=edges_list,
                 edge_attrs={'weight': weights},
                 vertex_attrs={'label': [int(x) for x in lines_nodes]})
    return G



### Hypergraph distance calculation function

In [10]:
# suplementary function to calculate particulare path length
def calc_path_weight(edges_to_counts_dict, line_graph, path):
    line_ids_nodes_dict = {i: line_graph.vs["label"][i] for i in range(len(line_graph.vs["label"]))}
    length = 0
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        e = line_graph.get_eid(start, end)
        length += line_graph.es[e]["weight"]
    length += (1 / 2) * (1 / edges_to_counts_dict[line_ids_nodes_dict[path[0]]] + \
                         1 / edges_to_counts_dict[line_ids_nodes_dict[path[-1]]])
    return length

### Line-graph generation

In [11]:
# if we constructed line-graph once it is saved into suplementary/linegraphs to fasten further calculation
line_graph = generate_weighted_line_graph(edges_to_nodes_dict, edges_to_counts_dict)

preparing data for line graph creation
539052
line graph creation


In [12]:
len(line_graph.vs), len(line_graph.es)

(2205, 539052)

In [17]:
from time import sleep


# функция для вычисления попарных расстояний
# на вход принимает (line_graph, points, pos)
# points - точки от которых надо посчитать расстояние до всех остальных
# pos - номер воркера (для tqdm)
# возвращает словарь dict[e1,e1]:distance
def do_calc(data):
    line_graph = data[0]
    points = data[1]
    pos = data[2]
    all_paths_length = {}
    sleep(max(pos, 0) / 10)
    print('start', pos)
    iter = tqdm(points, position=pos) if pos > 0 else points
    for v1 in iter:
        results_path = line_graph.get_shortest_paths(v1, output="epath", weights=line_graph.es["weight"])
        for i, results in enumerate(results_path):
            if len(results) > 0:
                # Add up the weights across all edges on the shortest path
                distance = 0
                for e in results:
                    distance += line_graph.es[e]["weight"]
                all_paths_length[v1, i] = distance
                all_paths_length[i, v1] = distance
                # print("Shortest weighted distance is: ", distance)
            else:
                # print(f"End node could not be reached! {v1} {v2}")
                pass
    return all_paths_length




Расчет всех расстояний

In [18]:
USE_JOB_LIB = True
WORKERS = 5
all_points = [i for i in range(len(line_graph.vs))]
data = [(line_graph, all_points[i::WORKERS], i if not USE_JOB_LIB else -1) for i in range(WORKERS)]

if WORKERS <= 1:
    res = []
    for d in data:
        res.append(do_calc(d))
elif USE_JOB_LIB:
    res = Parallel(n_jobs=WORKERS, verbose=10)(delayed(do_calc)(d) for d in data)
else:
    with Pool(WORKERS) as p:
        # res это список словарей вида {node_from: {node_to: length}}
        res = p.map(do_calc, data)
# мерж результатов воркеров
all_paths_length = {}
for r in res:
    for (u, v), d in r.items():
        all_paths_length[u, v] = d
del res, data, all_points

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   20.3s remaining:   30.4s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:   20.7s remaining:   13.8s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   21.3s finished


:### Computation of distances between specified pairs of nodes

In [19]:
# считаем self-loop (так как оно почему-то долго) и ребра
self_loops_weight = {}
all_edges = set()

for edges in tqdm(nodes_to_edges_dict.values()):
    for e in edges:
        all_edges.add(e)
        self_loops_weight[e] = line_graph.es["weight"][line_graph.get_eid(e, e)]

  0%|          | 0/135 [00:00<?, ?it/s]

In [20]:
# матрица расстояний
dst_mat = {}
all_edges_list = list(all_edges)
for i1 in trange(len(all_edges_list)):
    for i2 in range(i1, len(all_edges_list)):
        e1, e2 = all_edges_list[i1], all_edges_list[i2]
        if (e1, e2) in dst_mat:
            continue
        if e1 == e2:
            dst = self_loops_weight[e1]
            dst += 1 / edges_to_counts_dict[e1]
        else:
            dst = all_paths_length[e1, e2]  # расстояние в line graph
            dst += 1 / 2 / edges_to_counts_dict[e1] + 1 / 2 / edges_to_counts_dict[e2]
        dst_mat[e1, e2] = dst
        dst_mat[e2, e1] = dst
del all_edges_list

  0%|          | 0/2205 [00:00<?, ?it/s]

In [21]:
dists_dict = {}
paths_dict = {}
node_names = list(nodes_to_edges_dict.keys())
start_id = 0
final_id = len(node_names) - 1
# start_id = 120
# final_id = 120

file_path = pathlib.Path(f'{DIST_DIR}/dists_{data_type}_weighted-{start_id}-{final_id}.pickle')

# if os.path.exists(file_path):
#     pass
# with open(file_path, 'rb') as handle:
#     old_res, _ = pickle.load(handle)
# else:
if True:
    time_start = time.time()
    for id_from in trange(start_id, final_id + 1):
        node_from = node_names[id_from]
        dists = []
        paths = []
        for node in nodes_to_edges_dict.keys():
            if node != node_from:
                dst = float('inf')
                for e1 in nodes_to_edges_dict[node_from]:
                    for e2 in nodes_to_edges_dict[node]:
                        dst = min(dst, dst_mat[e1, e2])
                dists.append(dst)
            else:
                dists.append(0)
            dists_dict[node_from] = dists

    # we save calculated distances and corresponding shortest paths to fasten furter analysis
    with open(file_path, 'wb') as handle:
        pickle.dump((dists_dict, paths_dict), handle)
    time_term = time.time()
    print(f'Elapsed time {time_term - time_start}')

  0%|          | 0/135 [00:00<?, ?it/s]

Elapsed time 14.944987773895264


In [22]:
if data_type == '2000-2001':
    with open(f'../data/dists_2000-2001_weighted-0-134.pickle', 'rb') as handle:
        old_res, paths = pickle.load(handle)

    for k in old_res:
        for i in range(len(old_res[k])):
            delta = old_res[k][i] - dists_dict[k][i]
            if abs(delta) > 0.0001:
                print(f'err {old_res[k][i]:.2f}_{dists_dict[k][i]:.2f}, {k} {list(nodes_to_edges_dict.keys())[i]}')
    del old_res, paths

In [ ]:
# data frame containing calculated distances between every nodes pair
hyper_dist_df = pd.DataFrame(dists_dict)
hyper_dist_df.index = list(nodes_to_edges_dict.keys())
hyper_dist_df

### Weighted clique projection

In [ ]:
# function generating weighted clique projection
# weights = 1 / frequency of appearance
def make_clique_projection_data(edges_to_nodes_dict, edges_to_counts_dict):
    file_path = pathlib.Path(CLIQUE_PATH)

    if not file_path.exists():
        proj_edges_dict = {}
        for edge, nodes in edges_to_nodes_dict.items():
            for i in range(len(nodes) - 1):
                for j in range(i + 1, len(nodes)):
                    node_from = nodes[i]
                    node_to = nodes[j]
                    if not (((node_from, node_to) in proj_edges_dict.keys())
                            or ((node_to, node_from) in proj_edges_dict.keys())):
                        proj_edges_dict[(node_from, node_to)] = edges_to_counts_dict[edge]
                    else:
                        if ((node_from, node_to) in proj_edges_dict.keys()):
                            proj_edges_dict[(node_from, node_to)] += edges_to_counts_dict[edge]
                        else:
                            proj_edges_dict[(node_to, node_from)] += edges_to_counts_dict[edge]

        with open(CLIQUE_PATH, 'w') as handle:
            for (node_from, node_to), weight in proj_edges_dict.items():
                handle.write(f'{node_from} {node_to} {weight}')
                handle.write('\n')

In [ ]:
# first, create cluque projection if it wasn't created previously
make_clique_projection_data(edges_to_nodes_dict, edges_to_counts_dict)

# load and clique projection
proj_graph_file = CLIQUE_PATH
with open(proj_graph_file) as file:
    lines = file.readlines()
    edges_list = []
    weights_list = []
    nodes_dict = {}
    i = -1
    for line in lines:
        from_node, to_node, weight = line.split(" ")

        # from_node = int(from_node)
        # to_node = int(to_node)

        if not from_node in nodes_dict.keys():
            i += 1
            from_id = i
            nodes_dict[from_node] = i
        else:
            from_id = nodes_dict[from_node]
        if not to_node in nodes_dict.keys():
            i += 1
            to_id = i
            nodes_dict[to_node] = i
        else:
            to_id = nodes_dict[to_node]
        edges_list.append([from_id, to_id])
        weights_list.append(1 / float(weight))

proj_graph = ig.Graph(edges_list)
proj_graph.es["weight"] = weights_list

proj_dist_dict = {}
proj_dist_path = {}

# calculation of distances and paths in cluque projection

for from_node in dists_dict.keys():
    from_node_id = nodes_dict[from_node]
    dists = []
    paths_tmp = []
    for node in list(nodes_to_edges_dict.keys()):
        node_id = nodes_dict[node]
        paths = proj_graph.get_shortest_paths(from_node_id, node_id, weights=proj_graph.es["weight"], output="epath")
        paths_tmp.append(paths[0])
        distance = 0
        for e in paths[0]:
            distance += proj_graph.es[e]["weight"]
        dists.append(distance)

    proj_dist_dict[from_node] = dists
    proj_dist_path[from_node] = paths_tmp

# data frame containing dists in clique proj for every nodes pair
proj_dist_df = pd.DataFrame(proj_dist_dict)
proj_dist_df.index = list(nodes_to_edges_dict.keys())
proj_dist_df

### Analysis and comparison

In [ ]:
hyper_dist_df.to_csv(f"results_hypergraph_dist_{data_type}.csv")
proj_dist_df.to_csv(f"results_clique-proj_dist_{data_type}.csv")

In [ ]:
all_hg_dists = []
all_proj_dists = []

for i in range(len(node_names) - 1):
    for j in range(i + 1, len(node_names)):
        from_node = node_names[i]
        to_node = node_names[j]
        if not from_node == to_node:
            all_hg_dists.append(dists_dict[from_node][j])
            all_proj_dists.append(proj_dist_dict[from_node][j])
max_q = 10
hg_quantiles = [np.quantile(all_hg_dists, q / max_q) for q in range(1, max_q)]
proj_quantiles = [np.quantile(all_proj_dists, q / max_q) for q in range(1, max_q)]



In [ ]:
hg_quantiles

In [ ]:
proj_quantiles

In [ ]:
compare_dict = {
    "from node id": [],
    "to node id": [],
    "from node": [],
    "to node": [],
    "hypergraph distance": [],
    "projected distance": [],
    "diff": [],
    "hypergraph rank": [],
    "projected rank": [],
    "rank difference": []
}
for i in range(len(node_names) - 1):
    for j in range(i + 1, len(node_names)):
        from_node = node_names[i]
        to_node = node_names[j]
        hg_dist = dists_dict[from_node][j]
        proj_dist = proj_dist_dict[from_node][j]

        if hg_dist <= hg_quantiles[0]:
            compare_dict["hypergraph rank"].append(1)
        elif hg_dist > hg_quantiles[-1]:
            compare_dict["hypergraph rank"].append(max_q)
        else:
            for q in range(1, max_q - 1):
                if hg_dist > hg_quantiles[q - 1] and hg_dist <= hg_quantiles[q]:
                    compare_dict["hypergraph rank"].append(q + 1)

        if proj_dist <= proj_quantiles[0]:
            compare_dict["projected rank"].append(1)
        elif proj_dist > proj_quantiles[-1]:
            compare_dict["projected rank"].append(max_q)
        else:
            for q in range(1, max_q - 1):
                if proj_dist > proj_quantiles[q - 1] and proj_dist <= proj_quantiles[q]:
                    compare_dict["projected rank"].append(q + 1)

        compare_dict["from node id"].append(i)
        compare_dict["to node id"].append(j)
        compare_dict["from node"].append(from_node)
        compare_dict["to node"].append(to_node)
        compare_dict["hypergraph distance"].append(hg_dist)
        compare_dict["projected distance"].append(proj_dist)
        compare_dict["diff"].append(hg_dist - proj_dist)
        compare_dict["rank difference"].append(
            abs(compare_dict["projected rank"][-1] - compare_dict["hypergraph rank"][-1]))

In [ ]:
compare_df = pd.DataFrame(compare_dict)

In [ ]:
compare_df.sort_values(by=["rank difference"], ascending=False, inplace=True)
compare_df.to_csv(f"compare_{data_type}.csv", sep=",")

In [ ]:
compare_df

In [ ]:
quant_matrix = np.zeros(shape=(max_q, max_q))

N_pairs = len(compare_dict["hypergraph rank"])

for i in range(N_pairs):
    h_rank = compare_dict["hypergraph rank"][i]
    g_rank = compare_dict["projected rank"][i]
    quant_matrix[h_rank - 1, g_rank - 1] += 1 / N_pairs


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(quant_matrix, cmap="binary", )
plt.xticks(range(0, max_q), labels=range(1, max_q + 1))
plt.yticks(range(0, max_q), labels=range(1, max_q + 1))
plt.xlabel('hypergraph distance rank')
plt.ylabel('projected distance rank')
plt.colorbar()
plt.title(data_type[:4])
plt.savefig(f'../figures/ranks_dist_{data_type}.pdf')


In [ ]:
from_name = "q-fin.st"
to_name = "q-fin.rm"
from_id = 65
to_id = 67
dists_dict[from_name][to_id]

In [ ]:
proj_dist_dict[from_name][to_id]

In [ ]:
paths_dict[from_name][to_id]

In [ ]:

for edge in paths_dict[from_name][to_id][0]:
    string = ''
    for x in edges_to_nodes_dict[edge]:
        string += x + ' '
    print(f'{string} weight = {edges_to_counts_dict[edge]}')

In [ ]:
proj_dist_path[from_name][to_id]

In [ ]:
for edge in proj_dist_path[from_name][to_id]:
    print(f'weight = {1 / proj_graph.es[edge]["weight"]}')
    print(node_names[proj_graph.es[edge].source])
    print(node_names[proj_graph.es[edge].target])